In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import os
from tensorflow.keras.callbacks import ModelCheckpoint
import numpy as np
from tensorflow.keras.preprocessing import image

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
data_dir_testing = "/Users/aaryangulia/Downloads/archive-2/Test_Alphabet"
# Define the image size and batch size
img_size = (224, 224)
batch_size = 32

# Use ImageDataGenerator for data loading and augmentation
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,  # Rescale pixel values to [0, 1]
    validation_split=0.2,  # Split data into training and validation sets
    rotation_range=20,  # Randomly rotate images by up to 20 degrees
    width_shift_range=0.2,  # Randomly shift images horizontally by up to 20% of the width
    height_shift_range=0.2,  # Randomly shift images vertically by up to 20% of the height
    shear_range=0.2,  # Shear intensity (shear angle in radians)
    zoom_range=0.2,  # Randomly zoom images by up to 20%
    horizontal_flip=True,  # Randomly flip images horizontally
    fill_mode='nearest'  # Strategy for filling in newly created pixels
)

testing_generator = datagen.flow_from_directory(
    data_dir_testing,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',  # For multi-class classification
)

Found 2700 images belonging to 27 classes.


In [16]:
# Define a function to load and preprocess an image
def load_and_preprocess_image(image_path, target_size=(224, 224)):
    img = image.load_img(image_path, target_size=target_size)  # Load the image
    img_array = image.img_to_array(img)  # Convert image to numpy array
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Rescale pixel values to [0, 1] as done during training
    return img_array

# Load and preprocess the image
image_path = '/Users/aaryangulia/Downloads/archive-2/Train_Alphabet/S/0c1b4b06-610a-4374-a784-aac831e6ff71.rgb_0000.png'
processed_image = load_and_preprocess_image(image_path)

In [13]:
model = tf.keras.models.load_model('epoch_custom_model.h5',custom_objects={'KerasLayer': hub.KerasLayer})

In [14]:
print("\n Model summary: ")
print(model.summary())


 Model summary: 
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_3 (KerasLayer)  (None, 1001)              3540265   
                                                                 
 dense_2 (Dense)             (None, 29)                29058     
                                                                 
 activation_2 (Activation)   (None, 29)                0         
                                                                 
Total params: 3569323 (13.62 MB)
Trainable params: 29058 (113.51 KB)
Non-trainable params: 3540265 (13.51 MB)
_________________________________________________________________
None


In [17]:
# Make a prediction using the pre-trained model
predictions = model.predict(processed_image)

# Get the predicted class label
predicted_class_idx = np.argmax(predictions[0])

# Decode the class index to get the class label (assuming you have a list of class labels)
class_labels = list(testing_generator.class_indices.keys())
predicted_class_label = class_labels[predicted_class_idx]

# Get the confidence score of the prediction
confidence = predictions[0][predicted_class_idx]

# Print the results
print(f'Predicted class: {predicted_class_label}')
print(f'Confidence: {confidence:.2f}')

1/1 [==============================] - 0s 142ms/step
Predicted class: S
Confidence: 0.99


In [18]:
tf.keras.models.save_model(model, 'epoch2_custom_model')

INFO:tensorflow:Assets written to: epoch2_custom_model/assets


INFO:tensorflow:Assets written to: epoch2_custom_model/assets
